<a href="https://colab.research.google.com/github/pasumarthi/EVA/blob/master/Assignment16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

tf.enable_eager_execution()

In [0]:
filename_train = 'cifar-10-data/train.tfrecords' 
filename_test = 'cifar-10-data/validation.tfrecords' 
HEIGHT = 32
WIDTH = 32
DEPTH = 3

In [4]:
from google.colab import files
uploaded = files.upload()


Saving tfrecord.py to tfrecord.py


In [5]:
!ls

sample_data  tfrecord.py


In [6]:
! python tfrecord.py

Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.
Generating cifar-10-data/train.tfrecords


Generating cifar-10-data/validation.tfrecords
Generating cifar-10-data/eval.tfrecords
Done!


In [7]:
!ls -il cifar-10-data

total 349676
20054051 drwxr-xr-x 2 2156 1103      4096 Jun  4  2009 cifar-10-batches-py
20054050 -rw------- 1 root root 170498071 Sep 10 02:32 cifar-10-python.tar.gz
20054062 -rw-r--r-- 1 root root  31260000 Sep 10 02:32 eval.tfrecords
20054060 -rw-r--r-- 1 root root 125040000 Sep 10 02:32 train.tfrecords
20054061 -rw-r--r-- 1 root root  31260000 Sep 10 02:32 validation.tfrecords


In [0]:
# Hyperparameters
# batch_size = 128
num_classes = 10
l = 10
num_filter = 20
compression = 0.5
dropout_rate = 0.15
batch_size = 64
epochs = 100

In [0]:
def de_serialize(serialized_example):
    features = tf.parse_single_example(
    serialized_example,
    features={
    'image': tf.FixedLenFeature([], tf.string),
    'label': tf.FixedLenFeature([], tf.int64),
    })
    image = tf.decode_raw(features['image'], tf.uint8)
    image.set_shape([DEPTH * HEIGHT * WIDTH])
    image = tf.cast(
    tf.transpose(tf.reshape(image, [DEPTH, HEIGHT, WIDTH]), [1, 2, 0]),
    tf.float32)
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label, 10)
    return image, label

In [0]:
# -----------TRAIN TFRecords
dataset_train = tf.data.TFRecordDataset(filename_train).repeat(epochs)
dataset_train = dataset_train.map(de_serialize)
dataset_train = dataset_train.batch(batch_size)

# -------------TEST TFRecords
dataset_test = tf.data.TFRecordDataset(filename_test).repeat(1)
dataset_test = dataset_test.map(de_serialize)
dataset_test = dataset_test.batch(batch_size)


 """Parses a single tf.Example into image and label tensors."""
    # Dimensions of the images in the CIFAR-10 dataset.
    # See http://www.cs.toronto.edu/~kriz/cifar.html 

In [0]:
############################# MODEL ################################
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = tf.keras.layers.BatchNormalization()(temp)
        relu = tf.keras.layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = tf.keras.layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = tf.keras.layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = tf.keras.layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat        
    return temp
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = tf.keras.layers.BatchNormalization()(input)
    relu = tf.keras.layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = tf.keras.layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = tf.keras.layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = tf.keras.layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)    
    return avg
def output_layer(input):
    global compression
    BatchNorm = tf.keras.layers.BatchNormalization()(input)
    relu = tf.keras.layers.Activation('relu')(BatchNorm)
    AvgPooling = tf.keras.layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = tf.keras.layers.Flatten()(AvgPooling)
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(flat)    
    return output   



In [0]:
input = tf.keras.Input(shape=(HEIGHT, WIDTH, DEPTH,))
First_Conv2D = tf.keras.layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

model = tf.keras.Model(inputs=[input], outputs=[output])
sgd = tf.keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
                optimizer=sgd,
                metrics=['accuracy'])


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 20)   80          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 20)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
# for j,i in enumerate(dataset_train):
#   print("test",i[0].shape,i[1].shape,j)

In [16]:
for zz in range(1,epochs+1):
  # determine Loss function and Optimizer
  print("TrainEpochs number{}/{}".format(zz,epochs))
  model.fit_generator(dataset_train,steps_per_epoch=624,epochs=1,verbose=1)

  print("Test Epochs number{}/{}".format(zz,epochs))
  model.evaluate_generator(dataset_test,steps=156,verbose=1)
  print("  ")
  print("------------------------")

TrainEpochs number1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
624/624 [==============================] - 119s 190ms/step - loss: 1.7426 - acc: 0.3653
Test Epochs number1/100
156/156 [==============================] - 5s 34ms/step - loss: 1.7454 - acc: 0.3654
  
------------------------
TrainEpochs number2/100
624/624 [==============================] - 74s 119ms/step - loss: 1.3709 - acc: 0.4969
Test Epochs number2/100
156/156 [==============================] - 4s 25ms/step - loss: 2.2651 - acc: 0.3597
  
------------------------
TrainEpochs number3/100
624/624 [==============================] - 74s 118ms/step - loss: 1.1707 - acc: 0.5756
Test Epochs number3/100
156/156 [==============================] - 4s 24ms/step - loss: 1.2302 - acc: 0.5788
  
------------------------
TrainEpochs number4/100
624/624 [==============================] - 74s 119ms/step - loss: 1.0215 - acc: 0.6314
Test Epochs number4/100
156/156 [=================